In [5]:
import numpy as np
import pandas as pd
import sklearn.preprocessing as preprocessing


In [6]:
df = pd.read_csv(
    "datas/EURUSD_H1.csv",
    sep="\t",
    nrows=2**16,
    header=0,
    dtype={
        "open": np.float64,
        "close": np.float64,
        "tickvol": np.float64,
        "vol": np.float64,
    },
    parse_dates=[[0,1]],
    index_col=[0],
    names=["date","time","open","high","low","close","tickvol","vol","spread"])

In [7]:
prices = np.nan_to_num(np.array(df["open"]), posinf=0, neginf=0)
prices_diff = np.diff(prices[: -64])
x = np.reshape(prices_diff[: -64], (-1, 1))

In [8]:
scaler = preprocessing.RobustScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)
print(scaler.center_, scaler.scale_)
print(np.min(x_scaled), np.max(x_scaled))
x_scaled[:64]

[0.] [0.0014]
-17.78571428571619 21.642857142859523


array([[ 4.        ],
       [ 0.78571429],
       [ 2.14285714],
       [ 0.21428571],
       [-0.14285714],
       [ 0.        ],
       [ 0.21428571],
       [-1.42857143],
       [-0.85714286],
       [-1.85714286],
       [ 0.        ],
       [ 0.14285714],
       [-2.42857143],
       [ 0.21428571],
       [-0.07142857],
       [-0.64285714],
       [ 5.5       ],
       [ 0.07142857],
       [ 3.71428571],
       [ 0.78571429],
       [ 3.92857143],
       [-1.14285714],
       [-0.92857143],
       [-0.07142857],
       [-0.42857143],
       [ 1.64285714],
       [-0.64285714],
       [-0.85714286],
       [ 0.78571429],
       [-0.28571429],
       [-0.07142857],
       [-0.78571429],
       [ 0.42857143],
       [ 0.5       ],
       [-1.28571429],
       [ 3.21428571],
       [ 3.71428571],
       [-0.5       ],
       [-0.35714286],
       [ 1.21428571],
       [-2.14285714],
       [ 0.71428571],
       [-0.28571429],
       [-1.92857143],
       [ 1.28571429],
       [ 0

In [44]:
print(len(df.index))
df = df.asfreq(freq='5min')
df = df.fillna(method='ffill')
print(len(df.index))

39314
54935
